# Classify 3-D Lung Tumor Scans Based on Persistence Images

This will test a range of persistence imager parameters to find the best representation

In [1]:
import numpy as np
import matplotlib.pylab as plt
import math
import os
import gudhi as gd
import pandas as pd
import PersistenceImages.persistence_images as pimg

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

### Read in and combine datasets

In [2]:
#Imaging Data
rad_phom_0s = np.load('./Radiomics_Homology/rad_phom_0s.npy', allow_pickle = True)
rad_phom_1s = np.load('./Radiomics_Homology/rad_phom_1s.npy', allow_pickle = True)
rad_phom_2s = np.load('./Radiomics_Homology/rad_phom_2s.npy', allow_pickle = True)

radg_phom_0s = np.load('./Radiogenomics_Homology/radg_phom_0s.npy', allow_pickle = True)
radg_phom_1s = np.load('./Radiogenomics_Homology/radg_phom_1s.npy', allow_pickle = True)
radg_phom_2s = np.load('./Radiogenomics_Homology/radg_phom_1s.npy', allow_pickle = True)

In [3]:
#Clinical Data
#Radiomics clinical data
rad_clinical = pd.read_csv("rad_clinic.csv")
rad_clinical = rad_clinical.drop(rad_clinical.index[127]) #Tumor 128 has no segmentation

rad_histology = rad_clinical.Histology

#Radiogenomics clinical data
radg_clinical = pd.read_csv("radg_clinic.csv", skiprows = range(1,50))
radg_clinical = radg_clinical[0:146]
radg_clinical = radg_clinical.drop(radg_clinical.index[[8, 142]]) #9 and 143 have no segmentation

radg_clinical["Histology"] = radg_clinical['Histology'].str.lower()
radg_histology = radg_clinical.Histology

In [7]:
#Combine Datasets

#Identify indices we want to keep
rad_adeno = rad_histology == 'adenocarcinoma'
rad_squamous = rad_histology == 'squamous cell carcinoma'
radg_adeno = radg_histology == 'adenocarcinoma'
radg_squamous = radg_histology == 'squamous cell carcinoma'

#Select clinical data
rad_histology_adsq = np.array(rad_histology[rad_adeno | rad_squamous])
radg_histology_adsq = np.array(radg_histology[radg_adeno | radg_squamous])

#Select persistent homology data
rad_phom_0s_adsq = rad_phom_0s[rad_adeno | rad_squamous]
rad_phom_1s_adsq = rad_phom_1s[rad_adeno | rad_squamous]
rad_phom_2s_adsq = rad_phom_2s[rad_adeno | rad_squamous]

radg_phom_0s_adsq = radg_phom_0s[radg_adeno | radg_squamous]
radg_phom_1s_adsq = radg_phom_1s[radg_adeno | radg_squamous]
radg_phom_2s_adsq = radg_phom_2s[radg_adeno | radg_squamous]


histology_adsq = np.array(list(radg_histology_adsq) + list(rad_histology_adsq))
phom_0s_adsq = np.array(list(rad_phom_0s_adsq) + list(radg_phom_0s_adsq))
phom_1s_adsq = np.array(list(rad_phom_1s_adsq) + list(radg_phom_1s_adsq))
phom_2s_adsq = np.array(list(rad_phom_2s_adsq) + list(radg_phom_2s_adsq))

### Define functions to streamline parameter search

In [5]:
def CreateImager(pixel_size, sigma):
    pers_imager = pimg.PersistenceImager()
    pers_imager.pixel_size = pixel_size
    pers_imager.birth_range = (0,1)
    pers_imager.pers_range = (0,1)
    pers_imager.kernel_params['sigma'][0] = [sigma, 0]
    pers_imager.kernel_params['sigma'][1] = [0, sigma]
    return(pers_imager)
    
    
def HomologyToImageVector(phom_0, phom_1, phom_2, imager):
    
    pers_img_0 = imager.transform(phom_0, skew=True)
    pers_img_1 = imager.transform(phom_1, skew=True)
    pers_img_2 = imager.transform(phom_2, skew=True)
    
    pers_img_0 = np.resize(pers_img_0, (1, len(pers_img_0)**2))
    pers_img_1 = np.resize(pers_img_1, (1, len(pers_img_1)**2))
    if len(phom_2) == 0: #Lung 192 is why we can't have nice things
        pers_img_2 = np.zeros_like(pers_img_1)
    else:
        pers_img_2 = imager.transform(phom_2, skew=True)
        
    return(pers_img_0, pers_img_1, pers_img_2)


def AllPhomsToImages(phom_0s, phom_1s, phom_2s, imager):
    concatenated_images = []
    for i in range(len(phom_0s)):
        
        pimg_0, pimg_1, pimg_2 = HomologyToImageVector(phom_0s[i], phom_1s[i], phom_2s[i], imager)
        imgs = np.concatenate((pimg_0[0], pimg_1[0], pimg_2[0]), axis=0)
        concatenated_images.append(imgs)
        
    concatenated_images = np.array(concatenated_images)
    return(concatenated_images)

def ClassifyImages(images, histology):
    #First we need to shuffle this dataset since otherwise k-fold will magnify batch effects.
    images, histology = shuffle(images, histology, random_state = 10)

    scores = []
    y_pred = []

    clf_logreg = LogisticRegression(penalty = 'l1', solver='liblinear')

    cv = KFold(n_splits=5, shuffle=False)
    for train_index, test_index in cv.split(images):

        X_train, X_test = images[train_index], images[test_index]
        y_train, y_test = histology[train_index], histology[test_index]
        clf_logreg.fit(X_train, y_train)
        y_pred.append(list(clf_logreg.predict(X_test)))
        scores.append(clf_logreg.score(X_test, y_test))
    
    return(np.mean(scores))

def ParameterSearchInstance(pixel_size, sigma, phom_0s, phom_1s, phom_2s, histology):
    imager = CreateImager(pixel_size, sigma)
    images = AllPhomsToImages(phom_0s, phom_1s, phom_2s, imager)
    acc = ClassifyImages(images, histology)
    return(acc)

In [9]:
acc = ParameterSearchInstance(0.05, 0.001, phom_0s_adsq, phom_1s_adsq, phom_2s_adsq, histology_adsq)
print(acc)

0.5931798806479114
